# BACKPROP NINJA

In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [2]:
words = open('C:\\Users\\kaspa\\Desktop\\makemore_fun\\makemore-master\\names.txt', mode='r').read().split()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
# build the vocabulary of chars and mapping to (stoi) / from (itos) integers
chars = sorted(list(set(''.join(words))))

stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0

itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)

print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [4]:
# create the dataset split
block_size = 3 # context length for the next word prediction

def build_dataset(words):
    X,Y = [],[] 
    
    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix] #crop and append
            
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape,Y.shape)
    return X,Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

X_train,Y_train = build_dataset(words[:n1])
X_dev,Y_dev = build_dataset(words[n1:n2])
X_test,Y_test = build_dataset(words[n2:])

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [5]:
# utility function we will use later when comparing manual gradients to Pytorch gradients
def cmp(s,dt,t):
  ex = torch.all(dt == t.grad).item()
  app = torch.allclose(dt, t.grad)
  maxdiff =(dt -t.grad).abs().max().item()
  print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [6]:
# MLP
n_embd = 10
n_hidden = 64

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size,n_embd),                      generator=g)
#---------------------------------------------------------------------------------
W1 = torch.randn((n_embd * block_size,n_hidden),          generator=g) * (5/3) / ((n_embd * block_size )**0.5) 
b1 = torch.randn(n_hidden,                                generator=g) * 0.1 

W2 = torch.randn((n_hidden,vocab_size),                   generator=g) * 0.1 
b2 = torch.randn(vocab_size,                              generator=g) * 0.1 
#---------------------------------------------------------------------------------

#BATCHNORM PARAMETERS
#---------------------------------------------------------------------------------
bngain = torch.ones((1,n_hidden))*0.1 + 1
bnbias = torch.ones((1,n_hidden))*0.1
#bnmean_running = torch.ones((1,n_hidden))
#bnstd_running  = torch.ones((1,n_hidden))
#---------------------------------------------------------------------------------

parameters = [C, W1, b1, W2, b2, bngain,bnbias]
print(sum(p.nelement() for p in parameters))
for p in parameters:
    p.requires_grad = True

4137


In [7]:
batch_size = 32
n = batch_size # a shorter variable also, for convenience

# construct a minibatch
ix = torch.randint(0, X_train.shape[0], (batch_size,), generator=g)
Xb, Yb = X_train[ix], Y_train[ix] # batch X,Y

In [8]:
# forward pass, "chunkated" into smaller steps that are possible to backward one at a time

emb = C[Xb] # embed the characters into vectors
embcat = emb.view(emb.shape[0], -1) # concatenate the vectors

# Linear layer 1
hprebn = embcat @ W1 + b1 # hidden layer pre-activation
# BatchNorm layer
bnmeani = 1/n*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias
# Non-linearity
h = torch.tanh(hpreact) # hidden layer

# Linear layer 2
logits = h @ W2 + b2 # output layer
# cross entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

# PyTorch backward pass
for p in parameters:
  p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, # afaik there is no cleaner way
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
  t.retain_grad()
loss.backward()
loss

tensor(3.3603, grad_fn=<NegBackward0>)

In [17]:
# Exercise 1: backprop through the whole thing manually, 
# backpropagating through exactly all of the variables 
# as they are defined in the forward pass above, one by one

# ++++++++++++++++

# STEP 1 - loss -> logprobs
# loss = -logprobs[range(n), Yb].mean()

# Example: dloss/da 
# loss = -(a+b+c) / 3 -> dloss/da = -1/3

dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1.0/n

# Check with PyTorch:
cmp('logprobs', dlogprobs, logprobs)
# ++++++++++++++++

# STEP 2 - loss -> logprobs -> probs
# logprobs = probs.log()

# Example: log(x) -> dlog/dx = 1/x

dprobs = (1.0 / probs) * dlogprobs # CHAIN RULE

# Check with PyTorch:
cmp('probs', dprobs, probs)
# ++++++++++++++++

# STEP 3 - loss -> logprobs -> probs -> counts
# probs = counts * counts_sum_inv

# -----------------
# STEP 3.1. - loss -> logprobs -> probs -> counts_sum_inv
# Example: c = a * b -> dc/db = a

dcounts_sum_inv = (counts * dprobs).sum(1, keepdim = True) # CHAIN RULE

# Check with PyTorch:
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
# -----------------
# STEP 3.1.1 - loss -> logprobs -> probs -> counts_sum_inv -> counts_sum
# counts_sum_inv = counts_sum**-1

dcounts_sum = (-counts_sum**-2) * dcounts_sum_inv # CHAIN RULE

# Check with PyTorch:
cmp('counts_sum', dcounts_sum, counts_sum)
# -----------------

# STEP 3.2 - loss -> logprobs -> probs -> counts
# Example: c = a * b -> dc/da = b
dcounts = counts_sum_inv * dprobs # CHAIN RULE

# STEP 3.2.1 -loss -> logprobs -> probs -> counts_sum_inv -> counts_sum -> counts
dcounts += torch.ones_like(counts) * dcounts_sum

cmp('counts', dcounts, counts)
# -----------------
# cmp('norm_logits', dnorm_logits, norm_logits)
# cmp('logit_maxes', dlogit_maxes, logit_maxes)
# cmp('logits', dlogits, logits)
# cmp('h', dh, h)
# cmp('W2', dW2, W2)
# cmp('b2', db2, b2)
# cmp('hpreact', dhpreact, hpreact)
# cmp('bngain', dbngain, bngain)
# cmp('bnbias', dbnbias, bnbias)
# cmp('bnraw', dbnraw, bnraw)
# cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
# cmp('bnvar', dbnvar, bnvar)
# cmp('bndiff2', dbndiff2, bndiff2)
# cmp('bndiff', dbndiff, bndiff)
# cmp('bnmeani', dbnmeani, bnmeani)
# cmp('hprebn', dhprebn, hprebn)
# cmp('embcat', dembcat, embcat)
# cmp('W1', dW1, W1)
# cmp('b1', db1, b1)
# cmp('emb', demb, emb)
# cmp('C', dC, C)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
probs           | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0
counts          | exact: True  | approximate: True  | maxdiff: 0.0
